In [1]:
import cv2
from cvzone.HandTrackingModule import HandDetector

In [2]:
from time import sleep
import numpy as np
import cvzone
#import Controller
from pynput.keyboard import Controller
#from pynput.keyboard import Controller

In [3]:
cap = cv2.VideoCapture(0)
#cap=cv2.flip(cap,1)
cap.set(3, 1280)
cap.set(4, 720)

detector = HandDetector(detectionCon=0.8)
keys = [["Q", "W", "E", "R", "T", "Y", "U", "I", "O", "P"],
        ["A", "S", "D", "F", "G", "H", "J", "K", "L", ";"],
        ["Z", "X", "C", "V", "B", "N", "M", ",", ".", "/"]]
finalText = ""

keyboard = Controller()

In [4]:
def drawAll(img, buttonList):
    for button in buttonList:
        x, y = button.pos
        w, h = button.size
        cvzone.cornerRect(img, (button.pos[0], button.pos[1], button.size[0], button.size[1]),
                          20, rt=0)
        cv2.rectangle(img, button.pos, (x + w, y + h), (255, 0, 255), cv2.FILLED)
        cv2.putText(img, button.text, (x + 20, y + 65),
                    cv2.FONT_HERSHEY_PLAIN, 4, (255, 255, 255), 4)
    return img

In [5]:
class Button():
    def __init__(self, pos, text, size=[85, 85]):
        self.pos = pos
        self.size = size
        self.text = text


In [6]:
buttonList = []
for i in range(len(keys)):
    for j, key in enumerate(keys[i]):
        buttonList.append(Button([100 * j + 50, 100 * i + 50], key))

In [7]:
while True:
    success, img = cap.read()
    img = detector.findHands(img)
    lmList, bboxInfo = detector.findPosition(img)
    img = drawAll(img, buttonList)

    if lmList:
        for button in buttonList:
            x, y = button.pos
            w, h = button.size

            if x < lmList[8][0] < x + w and y < lmList[8][1] < y + h:
                cv2.rectangle(img, (x - 5, y - 5), (x + w + 5, y + h + 5), (175, 0, 175), cv2.FILLED)
                cv2.putText(img, button.text, (x + 20, y + 65),
                            cv2.FONT_HERSHEY_PLAIN, 4, (255, 255, 255), 4)
                l, _, _ = detector.findDistance(8, 12, img, draw=False)
                print(l)

                ## when clicked
                if l < 30:
                    keyboard.press(button.text)
                    cv2.rectangle(img, button.pos, (x + w, y + h), (0, 255, 0), cv2.FILLED)
                    cv2.putText(img, button.text, (x + 20, y + 65),
                                cv2.FONT_HERSHEY_PLAIN, 4, (255, 255, 255), 4)
                    finalText += button.text
                    sleep(0.15)

    cv2.rectangle(img, (50, 350), (700, 450), (175, 0, 175), cv2.FILLED)
    cv2.putText(img, finalText, (60, 430),
                cv2.FONT_HERSHEY_PLAIN, 5, (255, 255, 255), 5)

    cv2.imshow("Image", img)
    cv2.waitKey(1)

71.16881339463234
67.06713054842886
76.6550715869472
79.83107164506812
80.049984384758
91.54780172128656
71.00704190430693
56.0357029044876
41.23105625617661
40.459856648287825
50.44799302251775
53.665631459994955
53.53503525729669
40.85339643163099
42.720018726587654
48.33218389437828
47.4236228055175
46.1410879802373
55.94640292279746
87.0
94.92101980067429
67.72001181334805
69.26037828369117
73.38937252763509
58.189346103904626
60.40695324215582
148.06079832285116
148.00337833982033
69.83552104767314
47.75981574503821
46.61544808322665
50.08991914547277
46.6476151587624
64.4980619863884
69.06518659932803
65.0
63.25345840347387
57.384666941614284
70.45565981523414
198.6353442869622
67.11929677819934
40.36087214122114
27.202941017470888
15.524174696260024
16.970562748477143
17.804493814764854
19.79898987322333
